# Introduction

This is a brief introduction to using the refractivesqlite package. 

This package allows to create a SQLite database from the database refractiveindex.info, interpolate the refractive indices expressed as formulas, searching, and exporting materials to CSV.

# Create the database

Note: once created, you don't need to create the DB again!
In both methods you can specify the interpolation_points for the case when *n* is expressed as a formula and *k* is not defined.

## From url

In [ ]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
db = DB.Database(dbpath)
db.create_database_from_url()

## Check URL version

In [21]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
db = DB.Database(dbpath)
db.check_url_version()

Database file found at refractive.db
http://refractiveindex.info/download/database/rii-database-2016-01-31.zip


## Load a previous version
(For example, version 2015-07-05)

In [ ]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
db = DB.Database(dbpath)
db.create_database_from_url(riiurl="http://refractiveindex.info/download/database/rii-database-2015-07-05.zip")

## From local yml folder

In [ ]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
ymlpath = "database"
db = DB.Database(dbpath)
db.create_database_from_folder(ymlpath,interpolation_points=200)

# Load database

In [22]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
db = DB.Database(dbpath)

Database file found at refractive.db


# Searches

# List all pages

In [ ]:
db.search_pages()

## Search pages by term

In [23]:
db.search_pages("otanicar")

3 results found.
pageid	shelf	book	page	filepath	hasrefractive	hasextinction	rangeMin	rangeMax	points
463	organic	ethylene_glycol	Sani_and_Otanicar	organic\C2H6O2 - ethylene glycol\Sani_and_Otanicar.yml	1	1	0.185	2.8	66
466	organic	propylene_glycol	Otanicar	organic\C3H8O2 - propylene glycol\Otanicar.yml	1	1	0.434	0.656	66
1542	other	TherminolVP-1	Otanicar	heat transfer fluids\Therminol VP-1\Otanicar.yml	0	1	0.2	1.5	66


## Search pages by exact term

In [24]:
db.search_pages("au",exact=True)

15 results found.
pageid	shelf	book	page	filepath	hasrefractive	hasextinction	rangeMin	rangeMax	points
27	main	Au	Rakic	main\Au\Rakic.yml	1	1	0.2066	12.4	200
28	main	Au	Olmon-ev	main\Au\Olmon-ev.yml	1	1	0.3	24.93	448
29	main	Au	Olmon-sc	main\Au\Olmon-sc.yml	1	1	0.3	24.93	448
30	main	Au	Olmon-ts	main\Au\Olmon-ts.yml	1	1	0.3	24.93	448
31	main	Au	McPeak	main\Au\McPeak.yml	1	1	0.3	1.7	141
32	main	Au	Babar	main\Au\Babar.yml	1	1	0.2066	12.4	69
33	main	Au	Ordal	main\Au\Ordal.yml	1	1	0.667	286.0	52
34	main	Au	Windt	main\Au\Windt.yml	1	1	0.00236	0.12157	36
35	main	Au	Hagemann	main\Au\Hagemann.yml	1	1	8.266e-06	248.0	149
36	main	Au	Hagemann-2	main\Au\Hagemann-2.yml	1	1	0.003542	0.8266	124
37	main	Au	Johnson	main\Au\Johnson.yml	1	1	0.1879	1.937	49
38	main	Au	Lemarchand	main\Au\Lemarchand-3.96nm.yml	1	1	0.35	1.8	291
39	main	Au	Lemarchand	main\Au\Lemarchand-4.62nm.yml	1	1	0.35	1.8	291
40	main	Au	Lemarchand	main\Au\Lemarchand-5.77nm.yml	1	1	0.35	1.8	291
41	main	Au	Lemarchand	main\Au\Lemarchand-11.7n

## Search by PageId

In [25]:
db.search_id(1542)

pageid	shelf	book	page	filepath	hasrefractive	hasextinction	rangeMin	rangeMax	points
1542	other	TherminolVP-1	Otanicar	heat transfer fluids\Therminol VP-1\Otanicar.yml	0	1	0.2	1.5	66


## Search materials by *n* interval

In [26]:
db.search_n(n=0.3,delta_n=.001)

*Search n = 0.3 delta_n = 0.001
19 results found.
pageid|shelf|book|page|wavelength|n
(4, 'main', 'Ag', 'Hagemann', 0.3351, 0.3)
(9, 'main', 'Al', 'McPeak', 0.36, 0.300125667)
(29, 'main', 'Au', 'Olmon-sc', 1.56, 0.3003)
(41, 'main', 'Au', 'Lemarchand', 0.7, 0.30069838)
(41, 'main', 'Au', 'Lemarchand', 0.705, 0.30001489)
(41, 'main', 'Au', 'Lemarchand', 0.71, 0.29953164)
(41, 'main', 'Au', 'Lemarchand', 0.715, 0.29923605)
(41, 'main', 'Au', 'Lemarchand', 0.72, 0.29911651)
(41, 'main', 'Au', 'Lemarchand', 0.725, 0.29916233)
(41, 'main', 'Au', 'Lemarchand', 0.73, 0.29936366)
(41, 'main', 'Au', 'Lemarchand', 0.735, 0.29971138)
(41, 'main', 'Au', 'Lemarchand', 0.74, 0.30019707)
(41, 'main', 'Au', 'Lemarchand', 0.745, 0.30081292)
(100, 'main', 'Cu', 'Johnson', 0.6168, 0.3)
(100, 'main', 'Cu', 'Johnson', 0.892, 0.3)
(168, 'main', 'Li', 'Rasigni', 0.2362, 0.299)
(169, 'main', 'Li', 'Inagaki', 0.365, 0.301)
(197, 'main', 'Nb', 'Golovashkin-4.2', 2.5, 0.3)
(292, 'main', 'Pb', 'Golovashkin-4.2',

## Search materials by *k* interval

In [27]:
db.search_k(k=0.3,delta_k=.001)

*Search k = 0.3 delta_k = 0.001
41 results found.
pageid|shelf|book|page|wavelength|k
(4, 'main', 'Ag', 'Hagemann', 0.0253, 0.3)
(12, 'main', 'Al', 'Hagemann', 0.08551, 0.301)
(18, 'main', 'GaAs', 'Jellison', 0.53, 0.301)
(71, 'main', 'C', 'Djurisic-o', 0.042763, 0.300338)
(72, 'main', 'C', 'Djurisic-e', 0.031025, 0.300682)
(72, 'main', 'C', 'Djurisic-e', 0.031055, 0.299383)
(72, 'main', 'C', 'Djurisic-e', 0.034861, 0.299199)
(72, 'main', 'C', 'Djurisic-e', 0.034898, 0.300196)
(72, 'main', 'C', 'Djurisic-e', 0.049535, 0.300239)
(72, 'main', 'C', 'Djurisic-e', 0.04961, 0.299465)
(72, 'main', 'C', 'Djurisic-e', 0.057251, 0.30008)
(72, 'main', 'C', 'Djurisic-e', 0.136643, 0.299196)
(76, 'main', 'B4C', 'Larruquert', 0.898600363, 0.29904207)
(80, 'main', 'SiC', 'Larruquert', 0.608179276, 0.30014661)
(80, 'main', 'SiC', 'Larruquert', 20.0006755, 0.30097241)
(81, 'main', 'Ca', 'Rodriguez-de_Marcos', 0.038847598, 0.300510958)
(82, 'main', 'Ce', 'Fernandez-Perea', 0.06701848, 0.299016)
(102, 'm

## Search materials by *n,k* range

In [28]:
db.search_nk(n=0.3, delta_n=0.1,k=0.3,delta_k=0.1)

*Search n = 0.3 delta_n = 0.1 k = 0.3 delta_k = 0.1
42 results found.
pageid|shelf|book|page|wavelength|n|k
(71, 'main', 'C', 'Djurisic-o', 0.042069, 0.399493, 0.262777)
(71, 'main', 'C', 'Djurisic-o', 0.042126, 0.397854, 0.265747)
(71, 'main', 'C', 'Djurisic-o', 0.042183, 0.396244, 0.26875)
(71, 'main', 'C', 'Djurisic-o', 0.04224, 0.394662, 0.271782)
(71, 'main', 'C', 'Djurisic-o', 0.042298, 0.39311, 0.274845)
(71, 'main', 'C', 'Djurisic-o', 0.042355, 0.391588, 0.277937)
(71, 'main', 'C', 'Djurisic-o', 0.042413, 0.390097, 0.281058)
(71, 'main', 'C', 'Djurisic-o', 0.042471, 0.388638, 0.284207)
(71, 'main', 'C', 'Djurisic-o', 0.042529, 0.387211, 0.287382)
(71, 'main', 'C', 'Djurisic-o', 0.042588, 0.385817, 0.290584)
(71, 'main', 'C', 'Djurisic-o', 0.042646, 0.384457, 0.293811)
(71, 'main', 'C', 'Djurisic-o', 0.042705, 0.383131, 0.297063)
(71, 'main', 'C', 'Djurisic-o', 0.042763, 0.38184, 0.300338)
(71, 'main', 'C', 'Djurisic-o', 0.042822, 0.380584, 0.303635)
(71, 'main', 'C', 'Djurisic-

# Custom SQL search
You can search for custom sql queries (at your own risk). The function prints the number of results, and returns a list with n-tuples ordered by the fields on the select statement.

In [29]:
db.search_custom('select * from pages where shelf="main" and book="Ag" and page LIKE "%k%"')

2 results found.


[(0, 'main', 'Ag', 'Rakic', 'main\\Ag\\Rakic.yml', 1, 1, 0.2066, 12.4, 200),
 (1, 'main', 'Ag', 'McPeak', 'main\\Ag\\McPeak.yml', 1, 1, 0.3, 1.7, 141)]

In [30]:
db.search_custom('select wave,coeff from extcoeff where pageid = 1 and wave between 0.3 and 0.4')

11 results found.


[(0.3, 0.972336834),
 (0.31, 0.572501372),
 (0.32, 0.331538655),
 (0.33, 0.585441289),
 (0.34, 1.052042671),
 (0.35, 1.326866905),
 (0.36, 1.533000885),
 (0.37, 1.708000982),
 (0.38, 1.858729622),
 (0.39, 1.99546181),
 (0.4, 2.122943979)]

In [31]:
db.search_custom('select wave,refindex from refractiveindex where pageid = 1 and wave between 0.3 and 0.4')

11 results found.


[(0.3, 1.646857286),
 (0.31, 1.455629699),
 (0.32, 0.920058628),
 (0.33, 0.233797996),
 (0.34, 0.103893124),
 (0.35, 0.07599858),
 (0.36, 0.063180003),
 (0.37, 0.056279242),
 (0.38, 0.050752406),
 (0.39, 0.048660415),
 (0.4, 0.04572895)]

In [32]:
db.search_custom('''select p.filepath, r.wave,refindex,coeff 
                    from refractiveindex r inner join extcoeff e on r.pageid = e.pageid and r.wave = e.wave
                    inner join pages p on r.pageid = p.pageid
                    where r.wave = .301''')

12 results found.


[('main\\Co\\Johnson.yml', 0.301, 1.44, 2.25),
 ('main\\Cr\\Johnson.yml', 0.301, 1.53, 2.34),
 ('main\\Fe\\Johnson.yml', 0.301, 1.67, 2.0),
 ('main\\Mn\\Johnson.yml', 0.301, 1.86, 2.19),
 ('main\\Ni\\Johnson.yml', 0.301, 2.02, 2.18),
 ('main\\Pd\\Johnson.yml', 0.301, 1.2, 2.29),
 ('main\\Ti\\Johnson.yml', 0.301, 1.45, 2.46),
 ('main\\V\\Johnson.yml', 0.301, 2.07, 2.95),
 ('perovskite\\CH3NH3PbI3\\Leguy.yml', 0.301, 1.7499, 0.9508),
 ('perovskite\\CH3NH3PbI3\\Leguy-hydrated.yml', 0.301, 2.7371, 1.0209),
 ('main\\Fe\\Johnson.yml', 0.301, 1.67, 2.0),
 ('main\\Ti\\Johnson.yml', 0.301, 1.45, 2.46)]

## *Be careful*: Get ALL CSVs

In [ ]:
db.get_material_csv_all(outputfolder="all")

# Interact with materials

## Load material by Id

In [33]:
mat = db.get_material(1579)

Material other\soot\acetylene soot - Dalzell.yml loaded.


## Get material info

In [34]:
print(mat.get_page_info())

OrderedDict([('pageid', 1579), ('shelf', '3d'), ('book', 'misc'), ('page', 'acetylene_soot'), ('filepath', 'other\\soot\\acetylene soot - Dalzell.yml'), ('hasrefractive', 1), ('hasextinction', 1), ('rangeMin', 0.4358), ('rangeMax', 10.0), ('points', 13)])


## Get material *n*

In [35]:
import numpy as np
n = mat.get_complete_refractive()
print(n)

[[0.4358, 1.56], [0.45, 1.56], [0.55, 1.56], [0.65, 1.57], [0.8065, 1.57], [2.5, 2.31], [3.0, 2.62], [4.0, 2.74], [5.0, 2.88], [6.0, 3.22], [7.0, 3.49], [8.5, 4.22], [10.0, 4.8]]


## Get material *n* as numpy array

In [36]:
import numpy as np
n = np.array(mat.get_complete_refractive())
print(n)

[[  0.4358   1.56  ]
 [  0.45     1.56  ]
 [  0.55     1.56  ]
 [  0.65     1.57  ]
 [  0.8065   1.57  ]
 [  2.5      2.31  ]
 [  3.       2.62  ]
 [  4.       2.74  ]
 [  5.       2.88  ]
 [  6.       3.22  ]
 [  7.       3.49  ]
 [  8.5      4.22  ]
 [ 10.       4.8   ]]


## Get material *k*

In [37]:
k = mat.get_complete_extinction()
print(k)

[[0.4358, 0.46], [0.45, 0.48], [0.55, 0.46], [0.65, 0.44], [0.8065, 0.46], [2.5, 1.26], [3.0, 1.62], [4.0, 1.64], [5.0, 1.82], [6.0, 1.84], [7.0, 2.17], [8.5, 3.46], [10.0, 3.82]]


## Get material *k*

In [38]:
k = np.array(mat.get_complete_extinction())
print(k)

[[  0.4358   0.46  ]
 [  0.45     0.48  ]
 [  0.55     0.46  ]
 [  0.65     0.44  ]
 [  0.8065   0.46  ]
 [  2.5      1.26  ]
 [  3.       1.62  ]
 [  4.       1.64  ]
 [  5.       1.82  ]
 [  6.       1.84  ]
 [  7.       2.17  ]
 [  8.5      3.46  ]
 [ 10.       3.82  ]]


## Get n,k data as CSV

In [39]:
mat.to_csv(output="mat1.csv")

Wrote mat1(nk).csv
